# Comparison of pH profiles for Pyruvate Carboxylase
The python code below compares the pH profiles generated by the Alberty and Balanced Biochemical Reaction (Iotti, et al) methods for the pyruvate carboxylase reaction. The profile is reported in the manuscript as Table 1. This document is available as a Jupyter Notebook at https://github.com/wrcannon/CompositeReactionFreeEnergies/.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import re
import os
from IPython.core.display import display
pd.set_option('display.max_columns', None,'display.max_rows', None)
pd.set_option('display.max_colwidth', None)
#from ipynb_latex_setup import *
%matplotlib inline
T = 298.15
R = 8.314e-03
RT = R*T
display(RT)
N_avogadro = 6.022140857e+23

2.4788191

In [2]:
def molecular_partition_function(x):
    a = max(x)
    q_i = np.exp(x-a)
    sum_q_i = np.sum(q_i)
    log_q = (a + np.log(sum_q_i))
    return(log_q)

# Pyruvic acid <=> acetaldehyde + CO2

All free energies of formation in aqueous solution were obtained from http://equilibrator.weizmann.ac.il/, version 2.2 with source code repository commit hash f8bc4ca931f41ae08c5cf15b8945c1b1a85158d0, using the component contribution method ( Noor E, Haraldsdóttir HS, Milo R, Fleming RMT (2013) Consistent Estimation of Gibbs Energy Using Component Contributions. PLoS Comput Biol 9(7): e1003098. https://doi.org/10.1371/journal.pcbi.1003098)

In [3]:
mindex_fixed = [np.array(['H+','H2O']),np.array([1,1])]
dgzero = 'DGZERO'
chg = 'Charge'
isomer = 'Pseudoisomer Index'
fx = 'Mole Fraction'
coeff = 'Stoich. Coeff'

mindex1 = [np.array(['pyruvate','pyruvate','acetaldehyde','CO2']),np.array([1,2,1,1])]
reaction1 = pd.DataFrame(index=mindex1, columns=[chg, dgzero, fx])
reaction1.loc[('pyruvate'), chg] = np.array([0,-1])
reaction1.loc[('pyruvate'),coeff] = np.array([-1.0,-1.0])
reaction1.loc[('acetaldehyde'), chg] = np.float_(0)
reaction1.loc[('acetaldehyde'),fx] = np.float_(1.0)
reaction1.loc[('acetaldehyde'),coeff] = np.float_(1.0)
#Alberty RA. Thermodynamics of Biochemical Reactions (1st ed.). Hoboken, New Jersey: John Wiley & Sons
reaction1.loc[('acetaldehyde'),dgzero] = np.float_(-139.0) 
# Component Contribution via eQuilibrator
reaction1.loc[('acetaldehyde'),dgzero] = np.float_(-140.3) 
reaction1.loc[('CO2'), chg] = np.float_(0)
#Alberty RA. Thermodynamics of Biochemical Reactions (1st ed.). Hoboken, New Jersey: John Wiley & Sons
reaction1.loc[('CO2'),dgzero] = np.float_(-386.0) 
# Component Contribution:
reaction1.loc[('CO2'),dgzero] = np.float_(-386.0) 
reaction1.loc[('CO2'), fx] = np.float_(1)
reaction1.loc[('CO2'),coeff] = np.float_(1.0)
results = pd.DataFrame()

In [4]:
for pH in range(0,9):
    log_Hconc = -pH*np.log(10)
    fixed_metabolite = pd.DataFrame(index=mindex_fixed, columns=[chg, dgzero, fx])
    fixed_metabolite.loc[('H+',1),dgzero] = np.float_(RT*log_Hconc)
    fixed_metabolite.loc[('H+',1), chg] = np.float_(1.0)
    fixed_metabolite.loc[('H+',1),fx] = np.float_(1.0)



    #Martell AE, Smith RM, Motekaitis RJ. NIST Critically Selected Stability Constants of Metal Complexes Database, 8.0. NIST: Gaithersburg, MD, 2004
    reaction1.loc[('pyruvate',1),dgzero] = np.float_(-485.8) 
    reaction1.loc[('pyruvate',2),dgzero] = np.float_(-472.3) + fixed_metabolite.loc[('H+',1),dgzero] 
    # Component contribution via eQuilibrator:
    reaction1.loc[('pyruvate',1),dgzero] = np.float_(-483.6) 
    reaction1.loc[('pyruvate',2),dgzero] = np.float_(-466.9) + fixed_metabolite.loc[('H+',1),dgzero] 

    # Calculate molecular partition function for pyruvate
    x = np.array([-(reaction1.loc[('pyruvate',1),dgzero]/RT), \
         -((reaction1.loc[('pyruvate',2),dgzero]/RT))]) 
    log_q_pyr = molecular_partition_function(x)

    reaction1.loc[('pyruvate',1),fx] = np.exp(-reaction1.loc[('pyruvate',1),dgzero]/RT - log_q_pyr)                
    #reaction1.loc[('pyruvate',2),fx] = np.exp(-(reaction1.loc[('pyruvate',2),dgzero]+reaction1.loc[('H+',1),dgzero])/RT)/q
    reaction1.loc[('pyruvate',2),fx] = np.exp(-(reaction1.loc[('pyruvate',2),dgzero]/RT) - log_q_pyr)


    #BBR:
    dg0_bbr = np.sum(reaction1[dgzero].multiply(reaction1[fx].multiply(reaction1[coeff]))) 

    #Alberty
    dg0_a = reaction1.loc[('acetaldehyde',1),dgzero]*reaction1.loc[('acetaldehyde',1),coeff] + \
            reaction1.loc[('CO2',1),dgzero]*reaction1.loc[('CO2',1),coeff] + \
            (-1)*RT*log_q_pyr*reaction1.loc[('pyruvate',1),coeff]
    results.loc['BBR',pH] = dg0_bbr
    results.loc['Alberty',pH] = dg0_a
display(results)
    

,0,1,2,3,4,5,6,7,8
BBR,-42.719785,-42.828852,-43.260349,-42.470454,-37.045924,-30.960515,-25.168629,-19.448117,-13.738709
Alberty,-42.697062,-42.670772,-42.422155,-40.761260,-36.368589,-30.840729,-25.151760,-19.445948,-13.738444
